In [1]:
from shapely.geometry import Point
from collections import Counter
import seaborn as sns
import datetime as DT
import io
import re
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
mma_data = pd.read_csv('csv files/UFC.csv')

/opt/anaconda3/envs/geospatial/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (144,145) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
mma_data

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,...,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,R_index_ape,B_index_ape,R_index_ape2,B_index_ape2
0,Adrian Yanez,Gustavo Lopez,Chris Tognoni,3/20/2021,"Las Vegas, Nevada, USA",Red,False,Bantamweight,0.000,0.0,...,Orthodox,170.18,177.80,135.0,31.0,27.0,1.04,1.03,7.62,5.08
1,Trevin Giles,Roman Dolidze,Herb Dean,3/20/2021,"Las Vegas, Nevada, USA",Red,False,Middleweight,0.500,0.0,...,Orthodox,182.88,187.96,185.0,32.0,28.0,1.03,1.03,5.08,5.08
2,Tai Tuivasa,Harry Hunsucker,Herb Dean,3/20/2021,"Las Vegas, Nevada, USA",Red,False,Heavyweight,NaN,NaN,...,Southpaw,187.96,190.50,264.0,32.0,28.0,1.01,1.01,2.54,2.54
3,Cheyanne Buys,Montserrat Conejo,Mark Smith,3/20/2021,"Las Vegas, Nevada, USA",Blue,False,WomenStrawweight,NaN,NaN,...,Switch,160.02,160.02,115.0,28.0,25.0,1.0,1.02,0.00,2.54
4,Marion Reneau,Macy Chiasson,Mark Smith,3/20/2021,"Las Vegas, Nevada, USA",Blue,False,WomenBantamweight,0.125,0.0,...,Orthodox,167.64,172.72,135.0,29.0,43.0,1.03,1.01,5.08,2.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6007,Remco Pardoel,Alberta Cerra Leon,John McCarthy,3/11/1994,"Denver, Colorado, USA",Red,False,OpenWeight,NaN,NaN,...,Southpaw,190.50,NaN,260.0,NaN,NaN,0,0,-190.50,-172.72
6008,Orlando Wiet,Robert Lucarelli,John McCarthy,3/11/1994,"Denver, Colorado, USA",Red,False,OpenWeight,NaN,NaN,...,Southpaw,177.80,NaN,170.0,NaN,NaN,0,0,-177.80,-187.96
6009,Johnny Rhodes,David Levicki,John McCarthy,3/11/1994,"Denver, Colorado, USA",Red,False,OpenWeight,NaN,NaN,...,Orthodox,182.88,NaN,210.0,NaN,NaN,0,0,-182.88,-195.58
6010,Patrick Smith,Ray Wizard,John McCarthy,3/11/1994,"Denver, Colorado, USA",Red,False,OpenWeight,NaN,NaN,...,Orthodox,187.96,NaN,225.0,NaN,30.0,0,#DIV/0!,-187.96,0.00


In [4]:
mma_data.shape

(6012, 148)

In [5]:
for col in mma_data.columns:
    print(col)

R_fighter
B_fighter
Referee
date
location
Winner
title_bout
weight_class
B_avg_KD
B_avg_opp_KD
B_avg_SIG_STR_pct
B_avg_opp_SIG_STR_pct
B_avg_TD_pct
B_avg_opp_TD_pct
B_avg_SUB_ATT
B_avg_opp_SUB_ATT
B_avg_REV
B_avg_opp_REV
B_avg_SIG_STR_att
B_avg_SIG_STR_landed
B_avg_opp_SIG_STR_att
B_avg_opp_SIG_STR_landed
B_avg_TOTAL_STR_att
B_avg_TOTAL_STR_landed
B_avg_opp_TOTAL_STR_att
B_avg_opp_TOTAL_STR_landed
B_avg_TD_att
B_avg_TD_landed
B_avg_opp_TD_att
B_avg_opp_TD_landed
B_avg_HEAD_att
B_avg_HEAD_landed
B_avg_opp_HEAD_att
B_avg_opp_HEAD_landed
B_avg_BODY_att
B_avg_BODY_landed
B_avg_opp_BODY_att
B_avg_opp_BODY_landed
B_avg_LEG_att
B_avg_LEG_landed
B_avg_opp_LEG_att
B_avg_opp_LEG_landed
B_avg_DISTANCE_att
B_avg_DISTANCE_landed
B_avg_opp_DISTANCE_att
B_avg_opp_DISTANCE_landed
B_avg_CLINCH_att
B_avg_CLINCH_landed
B_avg_opp_CLINCH_att
B_avg_opp_CLINCH_landed
B_avg_GROUND_att
B_avg_GROUND_landed
B_avg_opp_GROUND_att
B_avg_opp_GROUND_landed
B_avg_CTRL_time(seconds)
B_avg_opp_CTRL_time(seconds)
B_total

In [6]:
mma_data2 = mma_data.dropna()

In [7]:
mma_data['date'] = pd.to_datetime(mma_data['date'])

mma_data2 = mma_data2[~(mma_data['date'] < '2000-01-01')]

/var/folders/yj/qq80gvk12fxg0xdlrjtwhxxr0000gn/T/ipykernel_46066/349260653.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  mma_data2 = mma_data2[~(mma_data['date'] < '2000-01-01')]


In [8]:
indexNames = mma_data2[ mma_data2['weight_class'] == 'WomenBantamweight' ].index

In [9]:
mma_data2.drop(indexNames, inplace = True)

In [10]:
indexNames2 = mma_data2[ mma_data2['weight_class'] == 'WomenStrawweight' ].index

In [11]:
mma_data2.drop(indexNames2, inplace = True)

In [12]:
indexNames3 = mma_data2[ mma_data2['weight_class'] == 'WomenFlyweight' ].index

In [13]:
mma_data2.drop(indexNames3, inplace = True)

In [14]:
indexNames4 = mma_data2[ mma_data2['weight_class'] == 'WomenFeatherweight' ].index

In [15]:
mma_data2.drop(indexNames4, inplace = True)

In [16]:
mma_data2

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,...,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,R_index_ape,B_index_ape,R_index_ape2,B_index_ape2
0,Adrian Yanez,Gustavo Lopez,Chris Tognoni,3/20/2021,"Las Vegas, Nevada, USA",Red,False,Bantamweight,0.000000,0.000,...,Orthodox,170.18,177.80,135.0,31.0,27.0,1.04,1.03,7.62,5.08
1,Trevin Giles,Roman Dolidze,Herb Dean,3/20/2021,"Las Vegas, Nevada, USA",Red,False,Middleweight,0.500000,0.000,...,Orthodox,182.88,187.96,185.0,32.0,28.0,1.03,1.03,5.08,5.08
5,Leonardo Santos,Grant Dawson,Chris Tognoni,3/20/2021,"Las Vegas, Nevada, USA",Blue,False,Lightweight,0.000000,0.000,...,Orthodox,182.88,190.50,155.0,27.0,41.0,1.04,1.03,7.62,5.08
6,Song Kenan,Max Griffin,Keith Peterson,3/20/2021,"Las Vegas, Nevada, USA",Blue,False,Welterweight,0.046875,0.125,...,Orthodox,182.88,180.34,170.0,35.0,31.0,0.99,1.07,-2.54,12.70
7,Derek Brunson,Kevin Holland,Herb Dean,3/20/2021,"Las Vegas, Nevada, USA",Red,False,Middleweight,0.156250,0.000,...,Southpaw,185.42,195.58,185.0,28.0,37.0,1.05,1.08,10.16,15.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5739,Jens Pulver,Dennis Hallman,Mario Yamasaki,9/28/2001,"Las Vegas, Nevada, USA",Red,True,Lightweight,0.000000,0.000,...,Southpaw,170.18,177.80,145.0,25.0,26.0,1.04,1.04,7.62,7.62
5744,Tito Ortiz,Elvis Sinosic,John McCarthy,6/29/2001,"East Rutherford, New Jersey, USA",Red,True,LightHeavyweight,0.000000,0.000,...,Orthodox,190.50,187.96,205.0,30.0,26.0,0.99,1.03,-2.54,5.08
5759,Tito Ortiz,Evan Tanner,John McCarthy,2/23/2001,"Atlantic City, New Jersey, USA",Red,True,LightHeavyweight,0.000000,0.000,...,Orthodox,190.50,187.96,205.0,30.0,26.0,0.99,1.03,-2.54,5.08
5767,Chuck Liddell,Jeff Monson,Mario Yamasaki,12/16/2000,"Tokyo, Japan",Red,False,Middleweight,0.000000,0.000,...,Orthodox,187.96,193.04,205.0,29.0,30.0,1.03,1.07,5.08,12.70


In [17]:
mma_data2.to_csv ('csv files/mma_data2.csv', index = False)

In [18]:
r_fighter_avg = mma_data2.drop(columns = ['B_fighter', 'B_avg_KD', 'B_avg_opp_KD', 'B_avg_SIG_STR_pct', 
                                         'B_avg_opp_SIG_STR_pct', 'B_avg_TD_pct', 'B_avg_opp_TD_pct', 
                                         'B_avg_SUB_ATT', 'B_avg_opp_SUB_ATT', 'B_avg_REV', 'B_avg_opp_REV', 
                                         'B_avg_SIG_STR_att', 'B_avg_SIG_STR_landed', 'B_avg_opp_SIG_STR_att', 
                                         'B_avg_opp_SIG_STR_landed', 'B_avg_TOTAL_STR_att', 'B_avg_TOTAL_STR_landed', 
                                         'B_avg_opp_TOTAL_STR_att', 'B_avg_opp_TOTAL_STR_landed', 'B_avg_TD_att', 
                                         'B_avg_TD_landed', 'B_avg_opp_TD_att', 'B_avg_opp_TD_landed', 'B_avg_HEAD_att', 
                                         'B_avg_HEAD_landed', 'B_avg_opp_HEAD_att', 'B_avg_opp_HEAD_landed', 
                                         'B_avg_BODY_att', 'B_avg_BODY_landed', 'B_avg_opp_BODY_att', 
                                         'B_avg_opp_BODY_landed', 'B_avg_LEG_att', 'B_avg_LEG_landed', 
                                         'B_avg_opp_LEG_att', 'B_avg_opp_LEG_landed', 'B_avg_DISTANCE_att', 
                                         'B_avg_DISTANCE_landed', 'B_avg_opp_DISTANCE_att', 'B_avg_opp_DISTANCE_landed', 
                                         'B_avg_CLINCH_att', 'B_avg_CLINCH_landed', 'B_avg_opp_CLINCH_att', 
                                         'B_avg_opp_CLINCH_landed', 'B_avg_GROUND_att', 'B_avg_GROUND_landed', 
                                         'B_avg_opp_GROUND_att', 'B_avg_opp_GROUND_landed', 'B_avg_CTRL_time(seconds)',
                                         'B_avg_opp_CTRL_time(seconds)', 'B_total_time_fought(seconds)', 
                                         'B_total_rounds_fought', 'B_total_title_bouts', 'B_current_win_streak', 
                                         'B_current_lose_streak', 'B_longest_win_streak', 'B_wins', 'B_losses', 
                                         'B_draw', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 
                                         'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission', 
                                         'B_win_by_TKO_Doctor_Stoppage', 'B_Stance', 'B_Height_cms', 'B_Reach_cms', 
                                         'B_Weight_lbs', 'B_age', 'B_index_ape', 'B_index_ape2'])

In [19]:
r_fighter_avg

,R_fighter,Referee,date,location,Winner,title_bout,weight_class,R_avg_KD,R_avg_opp_KD,R_avg_SIG_STR_pct,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age,R_index_ape,R_index_ape2
0,Adrian Yanez,Chris Tognoni,3/20/2021,"Las Vegas, Nevada, USA",Red,False,Bantamweight,1.000000,0.000000,0.500000,...,1,0,0,Orthodox,170.18,177.80,135.0,27.0,1.04,7.62
1,Trevin Giles,Herb Dean,3/20/2021,"Las Vegas, Nevada, USA",Red,False,Middleweight,1.031250,0.062500,0.576875,...,3,0,0,Orthodox,182.88,187.96,185.0,28.0,1.03,5.08
5,Leonardo Santos,Chris Tognoni,3/20/2021,"Las Vegas, Nevada, USA",Blue,False,Lightweight,0.312500,0.015625,0.565156,...,2,2,0,Orthodox,182.88,190.50,155.0,41.0,1.04,7.62
6,Song Kenan,Keith Peterson,3/20/2021,"Las Vegas, Nevada, USA",Blue,False,Welterweight,0.625000,0.000000,0.493125,...,3,0,0,Orthodox,182.88,180.34,170.0,31.0,0.99,-2.54
7,Derek Brunson,Herb Dean,3/20/2021,"Las Vegas, Nevada, USA",Red,False,Middleweight,0.024551,0.220703,0.515636,...,7,1,0,Southpaw,185.42,195.58,185.0,37.0,1.05,10.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5739,Jens Pulver,Mario Yamasaki,9/28/2001,"Las Vegas, Nevada, USA",Red,True,Lightweight,0.250000,0.000000,0.345625,...,2,0,0,Southpaw,170.18,177.80,145.0,26.0,1.04,7.62
5744,Tito Ortiz,John McCarthy,6/29/2001,"East Rutherford, New Jersey, USA",Red,True,LightHeavyweight,0.125000,0.250000,0.789766,...,3,1,1,Orthodox,190.50,187.96,205.0,26.0,0.99,-2.54
5759,Tito Ortiz,John McCarthy,2/23/2001,"Atlantic City, New Jersey, USA",Red,True,LightHeavyweight,0.250000,0.500000,0.579531,...,2,1,1,Orthodox,190.50,187.96,205.0,26.0,0.99,-2.54
5767,Chuck Liddell,Mario Yamasaki,12/16/2000,"Tokyo, Japan",Red,False,Middleweight,0.000000,0.000000,0.470000,...,0,0,1,Orthodox,187.96,193.04,205.0,30.0,1.03,5.08


In [20]:
r_fighter_avg.to_csv ('csv files/r_fighter_avg.csv', index = False)

In [21]:
r_fighter_avg2 = r_fighter_avg.drop(columns = ['Referee','date', 'location', 'Winner', 'title_bout'])

In [22]:
r_fighter_avg2

,R_fighter,weight_class,R_avg_KD,R_avg_opp_KD,R_avg_SIG_STR_pct,R_avg_opp_SIG_STR_pct,R_avg_TD_pct,R_avg_opp_TD_pct,R_avg_SUB_ATT,R_avg_opp_SUB_ATT,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age,R_index_ape,R_index_ape2
0,Adrian Yanez,Bantamweight,1.000000,0.000000,0.500000,0.460000,0.000000,0.000000,0.000000,0.000000,...,1,0,0,Orthodox,170.18,177.80,135.0,27.0,1.04,7.62
1,Trevin Giles,Middleweight,1.031250,0.062500,0.576875,0.381250,0.406250,0.116250,0.250000,1.187500,...,3,0,0,Orthodox,182.88,187.96,185.0,28.0,1.03,5.08
5,Leonardo Santos,Lightweight,0.312500,0.015625,0.565156,0.343594,0.337031,0.048906,0.039062,0.000000,...,2,2,0,Orthodox,182.88,190.50,155.0,41.0,1.04,7.62
6,Song Kenan,Welterweight,0.625000,0.000000,0.493125,0.430000,0.000000,0.687500,0.000000,0.250000,...,3,0,0,Orthodox,182.88,180.34,170.0,31.0,0.99,-2.54
7,Derek Brunson,Middleweight,0.024551,0.220703,0.515636,0.445974,0.355824,0.000000,0.000092,0.000000,...,7,1,0,Southpaw,185.42,195.58,185.0,37.0,1.05,10.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5739,Jens Pulver,Lightweight,0.250000,0.000000,0.345625,0.322500,0.062500,0.097500,0.187500,0.000000,...,2,0,0,Southpaw,170.18,177.80,145.0,26.0,1.04,7.62
5744,Tito Ortiz,LightHeavyweight,0.125000,0.250000,0.789766,0.223750,0.928906,0.000000,0.500000,0.164062,...,3,1,1,Orthodox,190.50,187.96,205.0,26.0,0.99,-2.54
5759,Tito Ortiz,LightHeavyweight,0.250000,0.500000,0.579531,0.447500,0.857812,0.000000,1.000000,0.328125,...,2,1,1,Orthodox,190.50,187.96,205.0,26.0,0.99,-2.54
5767,Chuck Liddell,Middleweight,0.000000,0.000000,0.470000,0.592500,0.250000,0.355000,0.000000,0.750000,...,0,0,1,Orthodox,187.96,193.04,205.0,30.0,1.03,5.08


In [23]:
for col in r_fighter_avg2.columns:
    print(col)

R_fighter
weight_class
R_avg_KD
R_avg_opp_KD
R_avg_SIG_STR_pct
R_avg_opp_SIG_STR_pct
R_avg_TD_pct
R_avg_opp_TD_pct
R_avg_SUB_ATT
R_avg_opp_SUB_ATT
R_avg_REV
R_avg_opp_REV
R_avg_SIG_STR_att
R_avg_SIG_STR_landed
R_avg_opp_SIG_STR_att
R_avg_opp_SIG_STR_landed
R_avg_TOTAL_STR_att
R_avg_TOTAL_STR_landed
R_avg_opp_TOTAL_STR_att
R_avg_opp_TOTAL_STR_landed
R_avg_TD_att
R_avg_TD_landed
R_avg_opp_TD_att
R_avg_opp_TD_landed
R_avg_HEAD_att
R_avg_HEAD_landed
R_avg_opp_HEAD_att
R_avg_opp_HEAD_landed
R_avg_BODY_att
R_avg_BODY_landed
R_avg_opp_BODY_att
R_avg_opp_BODY_landed
R_avg_LEG_att
R_avg_LEG_landed
R_avg_opp_LEG_att
R_avg_opp_LEG_landed
R_avg_DISTANCE_att
R_avg_DISTANCE_landed
R_avg_opp_DISTANCE_att
R_avg_opp_DISTANCE_landed
R_avg_CLINCH_att
R_avg_CLINCH_landed
R_avg_opp_CLINCH_att
R_avg_opp_CLINCH_landed
R_avg_GROUND_att
R_avg_GROUND_landed
R_avg_opp_GROUND_att
R_avg_opp_GROUND_landed
R_avg_CTRL_time(seconds)
R_avg_opp_CTRL_time(seconds)
R_total_time_fought(seconds)
R_total_rounds_fought
R_tota

In [24]:
r_fighter_avg2[r_fighter_avg2['R_fighter'] == 'Trevin Giles']

,R_fighter,weight_class,R_avg_KD,R_avg_opp_KD,R_avg_SIG_STR_pct,R_avg_opp_SIG_STR_pct,R_avg_TD_pct,R_avg_opp_TD_pct,R_avg_SUB_ATT,R_avg_opp_SUB_ATT,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age,R_index_ape,R_index_ape2
1,Trevin Giles,Middleweight,1.03125,0.0625,0.576875,0.38125,0.40625,0.11625,0.25,1.1875,...,3,0,0,Orthodox,182.88,187.96,185.0,28.0,1.03,5.08
184,Trevin Giles,Middleweight,0.06250,0.1250,0.513750,0.42250,0.31250,0.23250,0.50,2.3750,...,2,0,0,Orthodox,182.88,187.96,185.0,28.0,1.03,5.08
539,Trevin Giles,Middleweight,0.12500,0.2500,0.557500,0.29500,0.62500,0.21500,0.00,0.7500,...,2,0,0,Orthodox,182.88,187.96,185.0,27.0,1.03,5.08
788,Trevin Giles,Middleweight,0.25000,0.5000,0.585000,0.28000,0.25000,0.10000,0.00,0.5000,...,2,0,0,Orthodox,182.88,187.96,185.0,26.0,1.03,5.08


In [25]:
a = r_fighter_avg2.groupby(['R_fighter', 'R_Stance', 'weight_class']).mean()
print(a)

                                             R_avg_KD  R_avg_opp_KD  \
R_fighter            R_Stance weight_class                            
Aaron Phillips       Southpaw Bantamweight   0.000000      0.000000   
Aaron Riley          Southpaw Lightweight    0.000000      0.388021   
Aaron Simpson        Orthodox Middleweight   0.659375      0.000000   
                              Welterweight   0.755859      0.000000   
Abdul Razak Alhassan Orthodox Welterweight   1.552083      0.000000   
...                                               ...           ...   
Zak Cummings         Southpaw Welterweight   0.281250      0.000000   
Zelim Imadaev        Orthodox Welterweight   0.000000      0.000000   
Zhang Lipeng         Southpaw Lightweight    0.000000      0.000000   
Zhang Tiequan        Orthodox Featherweight  0.000000      0.000000   
Zubaira Tukhugov     Orthodox Featherweight  0.000000      0.000000   

                                             R_avg_SIG_STR_pct  \
R_fighter 

In [26]:
b = r_fighter_avg2.groupby(['R_fighter', 'R_Stance', 'weight_class'])['weight_class'].count()
print(b)

R_fighter             R_Stance  weight_class 
Aaron Phillips        Southpaw  Bantamweight     1
Aaron Riley           Southpaw  Lightweight      3
Aaron Simpson         Orthodox  Middleweight     5
                                Welterweight     2
Abdul Razak Alhassan  Orthodox  Welterweight     3
                                                ..
Zak Cummings          Southpaw  Welterweight     2
Zelim Imadaev         Orthodox  Welterweight     1
Zhang Lipeng          Southpaw  Lightweight      2
Zhang Tiequan         Orthodox  Featherweight    1
Zubaira Tukhugov      Orthodox  Featherweight    2
Name: weight_class, Length: 1103, dtype: int64


In [27]:
df_merged = a.merge(b, how = 'inner', left_index = True, right_index = True)

In [28]:
df_merged = df_merged.rename(columns = {'weight_class' : 'count_weight_class'})

In [29]:
df_merged = df_merged.reset_index()

In [30]:
r_df = df_merged

In [31]:
r_df

,R_fighter,R_Stance,weight_class,R_avg_KD,R_avg_opp_KD,R_avg_SIG_STR_pct,R_avg_opp_SIG_STR_pct,R_avg_TD_pct,R_avg_opp_TD_pct,R_avg_SUB_ATT,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age,R_index_ape2,count_weight_class
0,Aaron Phillips,Southpaw,Bantamweight,0.000000,0.000000,0.450000,0.540000,0.000000,0.620000,1.000000,...,0.000000,0.000000,0.0,0.0,175.26,180.34,135.0,25.0,5.08,1
1,Aaron Riley,Southpaw,Lightweight,0.000000,0.388021,0.377734,0.387943,0.233177,0.222005,0.210938,...,1.333333,0.000000,0.0,0.0,172.72,175.26,155.0,29.0,2.54,3
2,Aaron Simpson,Orthodox,Middleweight,0.659375,0.000000,0.554766,0.346094,0.567594,0.056375,0.418750,...,1.000000,1.800000,0.0,0.0,182.88,185.42,170.0,36.0,2.54,5
3,Aaron Simpson,Orthodox,Welterweight,0.755859,0.000000,0.608135,0.505996,0.155684,0.153984,0.199219,...,3.500000,2.000000,0.0,0.0,182.88,185.42,170.0,37.5,2.54,2
4,Abdul Razak Alhassan,Orthodox,Welterweight,1.552083,0.000000,0.500104,0.330521,0.076042,0.305208,0.000000,...,0.000000,2.666667,0.0,0.0,177.80,185.42,170.0,33.0,7.62,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,Zak Cummings,Southpaw,Welterweight,0.281250,0.000000,0.299531,0.499844,0.175781,0.174453,0.484375,...,2.000000,1.000000,1.5,0.0,182.88,190.50,185.0,32.0,7.62,2
1099,Zelim Imadaev,Orthodox,Welterweight,0.000000,0.000000,0.420000,0.510000,0.000000,0.540000,0.000000,...,0.000000,0.000000,0.0,0.0,182.88,193.04,170.0,24.0,10.16,1
1100,Zhang Lipeng,Southpaw,Lightweight,0.000000,0.000000,0.476250,0.438750,0.337500,0.110000,1.125000,...,1.000000,0.000000,0.0,0.0,180.34,180.34,155.0,24.5,0.00,2
1101,Zhang Tiequan,Orthodox,Featherweight,0.000000,0.000000,0.360000,0.330000,0.000000,0.000000,1.000000,...,0.000000,0.000000,1.0,0.0,172.72,175.26,155.0,33.0,2.54,1


In [32]:
r_df.to_csv ('csv files/r_df.csv', index = False)

In [33]:
for col in r_fighter_avg.columns:
    print(col)

R_fighter
Referee
date
location
Winner
title_bout
weight_class
R_avg_KD
R_avg_opp_KD
R_avg_SIG_STR_pct
R_avg_opp_SIG_STR_pct
R_avg_TD_pct
R_avg_opp_TD_pct
R_avg_SUB_ATT
R_avg_opp_SUB_ATT
R_avg_REV
R_avg_opp_REV
R_avg_SIG_STR_att
R_avg_SIG_STR_landed
R_avg_opp_SIG_STR_att
R_avg_opp_SIG_STR_landed
R_avg_TOTAL_STR_att
R_avg_TOTAL_STR_landed
R_avg_opp_TOTAL_STR_att
R_avg_opp_TOTAL_STR_landed
R_avg_TD_att
R_avg_TD_landed
R_avg_opp_TD_att
R_avg_opp_TD_landed
R_avg_HEAD_att
R_avg_HEAD_landed
R_avg_opp_HEAD_att
R_avg_opp_HEAD_landed
R_avg_BODY_att
R_avg_BODY_landed
R_avg_opp_BODY_att
R_avg_opp_BODY_landed
R_avg_LEG_att
R_avg_LEG_landed
R_avg_opp_LEG_att
R_avg_opp_LEG_landed
R_avg_DISTANCE_att
R_avg_DISTANCE_landed
R_avg_opp_DISTANCE_att
R_avg_opp_DISTANCE_landed
R_avg_CLINCH_att
R_avg_CLINCH_landed
R_avg_opp_CLINCH_att
R_avg_opp_CLINCH_landed
R_avg_GROUND_att
R_avg_GROUND_landed
R_avg_opp_GROUND_att
R_avg_opp_GROUND_landed
R_avg_CTRL_time(seconds)
R_avg_opp_CTRL_time(seconds)
R_total_time_foug

In [34]:
print(mma_data2[['R_fighter']])

            R_fighter
0        Adrian Yanez
1        Trevin Giles
5     Leonardo Santos
6          Song Kenan
7       Derek Brunson
...               ...
5739      Jens Pulver
5744       Tito Ortiz
5759       Tito Ortiz
5767    Chuck Liddell
5799       Tito Ortiz

[3578 rows x 1 columns]


In [35]:
R_fighter_list = mma_data2.index.tolist()
print(R_fighter_list)

[0, 1, 5, 6, 7, 11, 12, 14, 16, 17, 20, 21, 23, 24, 25, 26, 27, 28, 30, 31, 33, 36, 37, 38, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 57, 60, 61, 62, 63, 64, 66, 68, 71, 72, 73, 74, 75, 76, 77, 78, 80, 82, 85, 86, 87, 89, 90, 91, 93, 94, 95, 97, 98, 99, 100, 103, 107, 108, 109, 110, 111, 112, 113, 115, 116, 118, 119, 120, 121, 122, 123, 127, 128, 129, 131, 132, 133, 134, 135, 137, 139, 140, 141, 142, 143, 145, 147, 148, 150, 151, 152, 153, 155, 156, 157, 161, 162, 163, 165, 171, 172, 173, 174, 175, 176, 177, 178, 179, 181, 183, 184, 185, 187, 189, 190, 191, 193, 194, 195, 196, 199, 200, 201, 202, 204, 205, 206, 208, 211, 212, 214, 216, 217, 218, 220, 221, 223, 225, 228, 229, 230, 231, 234, 236, 240, 243, 244, 246, 247, 249, 250, 251, 252, 256, 257, 258, 260, 261, 262, 263, 266, 268, 270, 272, 276, 277, 279, 280, 282, 284, 286, 287, 288, 289, 290, 292, 296, 301, 302, 305, 306, 308, 309, 311, 312, 313, 315, 319, 322, 324, 325, 326, 329, 330, 333, 335, 336, 338, 339, 340, 341, 345, 

In [36]:
R_fighter = mma_data2["R_fighter"].tolist()
print(R_fighter)

['Adrian Yanez', 'Trevin Giles', 'Leonardo Santos', 'Song Kenan', 'Derek Brunson', 'Misha Cirkunov', 'Leon Edwards', 'Eryk Anders', 'Jonathan Martinez', 'Dan Ige', 'Matthew Semelsberger', 'Rani Yahya', 'Dominick Cruz', 'Song Yadong', 'Petr Yan', 'Jan Blachowicz', 'Thiago Santos', 'Joseph Benavidez', 'Islam Makhachev', 'Tim Elliott', 'Rogerio Bontorin', 'Sean Brady', 'Mario Bautista', 'Jairzinho Rozenstruik', 'Alexander Hernandez', 'Pedro Munhoz', 'Alex Caceres', 'Nikita Krylov', 'Dustin Jacoby', 'Aleksei Oleinik', 'Charles Rosa', 'Phil Hawes', 'Curtis Blaydes', 'Eddie Wineland', 'Andrei Arlovski', 'Jared Gordon', 'Nate Landwehr', 'Belal Muhammad', 'Maki Pitolo', 'Kamaru Usman', 'Ricky Simon', 'Kelvin Gastelum', 'Rodolfo Vieira', 'Andre Ewell', 'Devonte Smith', 'Michael Johnson', 'Alistair Overeem', 'Diego Ferreira', 'Cory Sandhagen', 'Mike Rodriguez', 'Ode Osbourne', 'SeungWoo Choi', 'Timur Valiev', 'Andrew Sanchez', 'Dustin Poirier', 'Brad Tavares', 'Arman Tsarukyan', 'Amir Albazi', '

In [37]:
R_fighter_count = Counter(R_fighter)
print(R_fighter_count)

Counter({'Jim Miller': 24, 'Demian Maia': 22, 'Donald Cerrone': 21, 'Diego Sanchez': 20, 'Mauricio Rua': 19, 'Michael Bisping': 19, 'Anderson Silva': 19, 'Andrei Arlovski': 17, 'Jon Jones': 17, 'Frankie Edgar': 17, 'Ross Pearson': 17, 'Joseph Benavidez': 16, 'Rafael Dos Anjos': 16, 'BJ Penn': 16, 'Lyoto Machida': 16, 'Georges St-Pierre': 16, 'Dustin Poirier': 15, 'Cub Swanson': 15, 'Matt Brown': 15, 'Thiago Alves': 15, 'Gleison Tibau': 15, 'Ovince Saint Preux': 14, 'Stefan Struve': 14, 'Edson Barboza': 14, 'Clay Guida': 14, 'Gray Maynard': 14, 'Rashad Evans': 14, 'Joe Lauzon': 14, 'Nate Diaz': 14, 'Alistair Overeem': 13, 'Jose Aldo': 13, 'Tony Ferguson': 13, 'Darren Elkins': 13, 'Jussier Formiga': 13, 'Glover Teixeira': 13, 'Court McGee': 13, 'Dennis Bermudez': 13, 'Jake Ellenberger': 13, 'Demetrious Johnson': 13, 'Johny Hendricks': 13, 'Michael Johnson': 12, 'Anthony Pettis': 12, 'Derrick Lewis': 12, 'Francisco Trinaldo': 12, 'Thales Leites': 12, 'Mike Pyle': 12, 'Nate Marquardt': 12,

In [38]:
R_fighter_df = pd.DataFrame.from_dict(R_fighter_count, orient = 'index').reset_index()
R_fighter_df

,index,0
0,Adrian Yanez,1
1,Trevin Giles,4
2,Leonardo Santos,6
3,Song Kenan,3
4,Derek Brunson,9
...,...,...
925,Trevor Prangley,1
926,Matt Lindland,4
927,Evan Tanner,4
928,Frank Trigg,1


In [39]:
R_fighter_df.columns = ['name', 'count']
print(R_fighter_df)

                name  count
0       Adrian Yanez      1
1       Trevin Giles      4
2    Leonardo Santos      6
3         Song Kenan      3
4      Derek Brunson      9
..               ...    ...
925  Trevor Prangley      1
926    Matt Lindland      4
927      Evan Tanner      4
928      Frank Trigg      1
929      Jens Pulver      2

[930 rows x 2 columns]


In [40]:
R_fighter_df.to_csv ('csv files/R_fighter_df.csv', index = False)

In [41]:
print(mma_data2[['B_fighter']])

            B_fighter
0       Gustavo Lopez
1       Roman Dolidze
5        Grant Dawson
6         Max Griffin
7       Kevin Holland
...               ...
5739   Dennis Hallman
5744    Elvis Sinosic
5759      Evan Tanner
5767      Jeff Monson
5799  Wanderlei Silva

[3578 rows x 1 columns]


In [42]:
B_fighter_list = mma_data2.index.tolist()
print(B_fighter_list)

[0, 1, 5, 6, 7, 11, 12, 14, 16, 17, 20, 21, 23, 24, 25, 26, 27, 28, 30, 31, 33, 36, 37, 38, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 57, 60, 61, 62, 63, 64, 66, 68, 71, 72, 73, 74, 75, 76, 77, 78, 80, 82, 85, 86, 87, 89, 90, 91, 93, 94, 95, 97, 98, 99, 100, 103, 107, 108, 109, 110, 111, 112, 113, 115, 116, 118, 119, 120, 121, 122, 123, 127, 128, 129, 131, 132, 133, 134, 135, 137, 139, 140, 141, 142, 143, 145, 147, 148, 150, 151, 152, 153, 155, 156, 157, 161, 162, 163, 165, 171, 172, 173, 174, 175, 176, 177, 178, 179, 181, 183, 184, 185, 187, 189, 190, 191, 193, 194, 195, 196, 199, 200, 201, 202, 204, 205, 206, 208, 211, 212, 214, 216, 217, 218, 220, 221, 223, 225, 228, 229, 230, 231, 234, 236, 240, 243, 244, 246, 247, 249, 250, 251, 252, 256, 257, 258, 260, 261, 262, 263, 266, 268, 270, 272, 276, 277, 279, 280, 282, 284, 286, 287, 288, 289, 290, 292, 296, 301, 302, 305, 306, 308, 309, 311, 312, 313, 315, 319, 322, 324, 325, 326, 329, 330, 333, 335, 336, 338, 339, 340, 341, 345, 

In [43]:
B_fighter = mma_data2["B_fighter"].tolist()
print(B_fighter)

['Gustavo Lopez', 'Roman Dolidze', 'Grant Dawson', 'Max Griffin', 'Kevin Holland', 'Ryan Spann', 'Belal Muhammad', 'Darren Stewart', 'Davey Grant', 'Gavin Tucker', 'Jason Witt', 'Ray Rodriguez', 'Casey Kenney', 'Kyler Phillips', 'Aljamain Sterling', 'Israel Adesanya', 'Aleksandar Rakic', 'Askar Askarov', 'Drew Dober', 'Jordan Espinosa', 'Kai Kara-France', 'Jake Matthews', 'Trevin Jones', 'Ciryl Gane', 'Thiago Moises', 'Jimmie Rivera', 'Kevin Croom', 'Magomed Ankalaev', 'Maxim Grishin', 'Chris Daukaus', 'Darrick Minner', 'Nassourdine Imavov', 'Derrick Lewis', 'John Castaneda', 'Tom Aspinall', 'Danny Chavez', 'Julian Erosa', 'Dhiego Lima', 'Julian Marquez', 'Gilbert Burns', 'Brian Kelleher', 'Ian Heinisch', 'Anthony Hernandez', 'Chris Gutierrez', 'Justin Jaynes', 'Clay Guida', 'Alexander Volkov', 'Beneil Dariush', 'Frankie Edgar', 'Danilo Marques', 'Jerome Rivera', 'Youssef Zalal', 'Martin Day', 'Makhmud Muradov', 'Conor McGregor', 'Antonio Carlos Junior', 'Matt Frevola', 'Zhalgas Zhumag

In [44]:
B_fighter_count = Counter(B_fighter)
print(B_fighter_count)

Counter({'Charles Oliveira': 18, 'Jeremy Stephens': 18, 'Nik Lentz': 14, 'Rafael Dos Anjos': 13, 'Evan Dunham': 12, 'Clay Guida': 11, 'Tim Means': 11, 'Sam Alvey': 11, 'Donald Cerrone': 11, 'Kevin Lee': 11, 'Gleison Tibau': 11, 'Nate Marquardt': 11, 'Tim Boetsch': 11, 'Dan Henderson': 11, 'Rick Story': 11, 'Andre Fili': 10, 'Kelvin Gastelum': 10, 'Jorge Masvidal': 10, 'Ben Saunders': 10, 'Nate Diaz': 10, 'Yancy Medeiros': 10, 'Chris Lytle': 10, 'Drew Dober': 9, 'Neil Magny': 9, 'Matt Brown': 9, 'Sean Strickland': 9, 'Niko Price': 9, 'Rogerio Nogueira': 9, 'Dan Hooker': 9, 'Junior Dos Santos': 9, 'Mike Perry': 9, 'Ed Herman': 9, 'Paul Felder': 9, 'Thiago Santos': 9, 'Anthony Smith': 9, 'Michael Johnson': 9, 'Jim Miller': 9, 'Joshua Burkman': 9, 'Stephen Thompson': 9, 'Beneil Dariush': 8, 'Frankie Edgar': 8, 'Marlon Vera': 8, 'Walt Harris': 8, 'Jan Blachowicz': 8, 'Max Holloway': 8, 'Anthony Rocco Martin': 8, 'Nikita Krylov': 8, 'Yoel Romero': 8, 'Omari Akhmedov': 8, 'Darren Elkins': 8, 

In [45]:
B_fighter_df = pd.DataFrame.from_dict(B_fighter_count, orient = 'index').reset_index()
B_fighter_df

,index,0
0,Gustavo Lopez,1
1,Roman Dolidze,1
2,Grant Dawson,2
3,Max Griffin,7
4,Kevin Holland,2
...,...,...
1086,Jeff Monson,2
1087,Ken Shamrock,4
1088,Nick Thompson,1
1089,Trevor Prangley,2


In [46]:
B_fighter_df.columns = ['name', 'count']
print(B_fighter_df)

                 name  count
0       Gustavo Lopez      1
1       Roman Dolidze      1
2        Grant Dawson      2
3         Max Griffin      7
4       Kevin Holland      2
...               ...    ...
1086      Jeff Monson      2
1087     Ken Shamrock      4
1088    Nick Thompson      1
1089  Trevor Prangley      2
1090   Alex Karalexis      1

[1091 rows x 2 columns]


In [47]:
B_fighter_df.to_csv ('csv files/B_fighter_df.csv', index = False)

In [48]:
b_fighter_avg = mma_data2.drop(columns = ['R_fighter', 'R_avg_KD', 'R_avg_opp_KD', 'R_avg_SIG_STR_pct', 
                                         'R_avg_opp_SIG_STR_pct', 'R_avg_TD_pct', 'R_avg_opp_TD_pct', 
                                         'R_avg_SUB_ATT', 'R_avg_opp_SUB_ATT', 'R_avg_REV', 'R_avg_opp_REV', 
                                         'R_avg_SIG_STR_att', 'R_avg_SIG_STR_landed', 'R_avg_opp_SIG_STR_att', 
                                         'R_avg_opp_SIG_STR_landed', 'R_avg_TOTAL_STR_att', 'R_avg_TOTAL_STR_landed', 
                                         'R_avg_opp_TOTAL_STR_att', 'R_avg_opp_TOTAL_STR_landed', 'R_avg_TD_att', 
                                         'R_avg_TD_landed', 'R_avg_opp_TD_att', 'R_avg_opp_TD_landed', 'R_avg_HEAD_att', 
                                         'R_avg_HEAD_landed', 'R_avg_opp_HEAD_att', 'R_avg_opp_HEAD_landed', 
                                         'R_avg_BODY_att', 'R_avg_BODY_landed', 'R_avg_opp_BODY_att', 
                                         'R_avg_opp_BODY_landed', 'R_avg_LEG_att', 'R_avg_LEG_landed', 
                                         'R_avg_opp_LEG_att', 'R_avg_opp_LEG_landed', 'R_avg_DISTANCE_att', 
                                         'R_avg_DISTANCE_landed', 'R_avg_opp_DISTANCE_att', 'R_avg_opp_DISTANCE_landed', 
                                         'R_avg_CLINCH_att', 'R_avg_CLINCH_landed', 'R_avg_opp_CLINCH_att', 
                                         'R_avg_opp_CLINCH_landed', 'R_avg_GROUND_att', 'R_avg_GROUND_landed', 
                                         'R_avg_opp_GROUND_att', 'R_avg_opp_GROUND_landed', 'R_avg_CTRL_time(seconds)',
                                         'R_avg_opp_CTRL_time(seconds)', 'R_total_time_fought(seconds)', 
                                         'R_total_rounds_fought', 'R_total_title_bouts', 'R_current_win_streak', 
                                         'R_current_lose_streak', 'R_longest_win_streak', 'R_wins', 'R_losses', 
                                         'R_draw', 'R_win_by_Decision_Majority', 'R_win_by_Decision_Split', 
                                         'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission', 
                                         'R_win_by_TKO_Doctor_Stoppage', 'R_Stance', 'R_Height_cms', 'R_Reach_cms', 
                                         'R_Weight_lbs', 'R_age', 'R_index_ape', 'R_index_ape2'])

In [49]:
b_fighter_avg

,B_fighter,Referee,date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,...,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,B_age,B_index_ape,B_index_ape2
0,Gustavo Lopez,Chris Tognoni,3/20/2021,"Las Vegas, Nevada, USA",Red,False,Bantamweight,0.000000,0.000,0.420000,...,0,1,0,Orthodox,165.10,170.18,135.0,31.0,1.03,5.08
1,Roman Dolidze,Herb Dean,3/20/2021,"Las Vegas, Nevada, USA",Red,False,Middleweight,0.500000,0.000,0.660000,...,1,0,0,Orthodox,187.96,193.04,205.0,32.0,1.03,5.08
5,Grant Dawson,Chris Tognoni,3/20/2021,"Las Vegas, Nevada, USA",Blue,False,Lightweight,0.000000,0.000,0.515000,...,0,2,0,Switch,177.80,182.88,145.0,27.0,1.03,5.08
6,Max Griffin,Keith Peterson,3/20/2021,"Las Vegas, Nevada, USA",Blue,False,Welterweight,0.046875,0.125,0.459277,...,2,0,0,Orthodox,180.34,193.04,170.0,35.0,1.07,12.70
7,Kevin Holland,Herb Dean,3/20/2021,"Las Vegas, Nevada, USA",Red,False,Middleweight,0.156250,0.000,0.791582,...,3,2,0,Orthodox,190.50,205.74,185.0,28.0,1.08,15.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5739,Dennis Hallman,Mario Yamasaki,9/28/2001,"Las Vegas, Nevada, USA",Red,True,Lightweight,0.000000,0.000,0.000000,...,0,1,0,Orthodox,175.26,182.88,155.0,25.0,1.04,7.62
5744,Elvis Sinosic,John McCarthy,6/29/2001,"East Rutherford, New Jersey, USA",Red,True,LightHeavyweight,0.000000,0.000,0.400000,...,0,1,0,Orthodox,190.50,195.58,205.0,30.0,1.03,5.08
5759,Evan Tanner,John McCarthy,2/23/2001,"Atlantic City, New Jersey, USA",Red,True,LightHeavyweight,0.000000,0.000,0.737500,...,2,1,0,Orthodox,182.88,187.96,185.0,30.0,1.03,5.08
5767,Jeff Monson,Mario Yamasaki,12/16/2000,"Tokyo, Japan",Red,False,Middleweight,0.000000,0.000,0.500000,...,0,0,0,Orthodox,175.26,187.96,247.0,29.0,1.07,12.70


In [50]:
b_fighter_avg.to_csv ('csv files/b_fighter_avg.csv', index = False)

In [51]:
for col in b_fighter_avg.columns:
    print(col)

B_fighter
Referee
date
location
Winner
title_bout
weight_class
B_avg_KD
B_avg_opp_KD
B_avg_SIG_STR_pct
B_avg_opp_SIG_STR_pct
B_avg_TD_pct
B_avg_opp_TD_pct
B_avg_SUB_ATT
B_avg_opp_SUB_ATT
B_avg_REV
B_avg_opp_REV
B_avg_SIG_STR_att
B_avg_SIG_STR_landed
B_avg_opp_SIG_STR_att
B_avg_opp_SIG_STR_landed
B_avg_TOTAL_STR_att
B_avg_TOTAL_STR_landed
B_avg_opp_TOTAL_STR_att
B_avg_opp_TOTAL_STR_landed
B_avg_TD_att
B_avg_TD_landed
B_avg_opp_TD_att
B_avg_opp_TD_landed
B_avg_HEAD_att
B_avg_HEAD_landed
B_avg_opp_HEAD_att
B_avg_opp_HEAD_landed
B_avg_BODY_att
B_avg_BODY_landed
B_avg_opp_BODY_att
B_avg_opp_BODY_landed
B_avg_LEG_att
B_avg_LEG_landed
B_avg_opp_LEG_att
B_avg_opp_LEG_landed
B_avg_DISTANCE_att
B_avg_DISTANCE_landed
B_avg_opp_DISTANCE_att
B_avg_opp_DISTANCE_landed
B_avg_CLINCH_att
B_avg_CLINCH_landed
B_avg_opp_CLINCH_att
B_avg_opp_CLINCH_landed
B_avg_GROUND_att
B_avg_GROUND_landed
B_avg_opp_GROUND_att
B_avg_opp_GROUND_landed
B_avg_CTRL_time(seconds)
B_avg_opp_CTRL_time(seconds)
B_total_time_foug

In [52]:
b_fighter_avg2 = b_fighter_avg.drop(columns = ['Referee','date', 'location', 'Winner', 'title_bout'])

In [53]:
b_fighter_avg2

,B_fighter,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,...,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,B_age,B_index_ape,B_index_ape2
0,Gustavo Lopez,Bantamweight,0.000000,0.000,0.420000,0.495000,0.330000,0.360000,0.500000,1.000000,...,0,1,0,Orthodox,165.10,170.18,135.0,31.0,1.03,5.08
1,Roman Dolidze,Middleweight,0.500000,0.000,0.660000,0.305000,0.300000,0.500000,1.500000,0.000000,...,1,0,0,Orthodox,187.96,193.04,205.0,32.0,1.03,5.08
5,Grant Dawson,Lightweight,0.000000,0.000,0.515000,0.473750,0.435000,0.250000,1.250000,0.625000,...,0,2,0,Switch,177.80,182.88,145.0,27.0,1.03,5.08
6,Max Griffin,Welterweight,0.046875,0.125,0.459277,0.404687,0.322188,0.068867,0.189453,0.000000,...,2,0,0,Orthodox,180.34,193.04,170.0,35.0,1.07,12.70
7,Kevin Holland,Middleweight,0.156250,0.000,0.791582,0.850586,0.269199,0.393398,0.029297,0.080078,...,3,2,0,Orthodox,190.50,205.74,185.0,28.0,1.08,15.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5739,Dennis Hallman,Lightweight,0.000000,0.000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,...,0,1,0,Orthodox,175.26,182.88,155.0,25.0,1.04,7.62
5744,Elvis Sinosic,LightHeavyweight,0.000000,0.000,0.400000,0.400000,0.000000,0.000000,2.000000,0.000000,...,0,1,0,Orthodox,190.50,195.58,205.0,30.0,1.03,5.08
5759,Evan Tanner,LightHeavyweight,0.000000,0.000,0.737500,0.475000,1.000000,0.250000,1.250000,0.000000,...,2,1,0,Orthodox,182.88,187.96,185.0,30.0,1.03,5.08
5767,Jeff Monson,Middleweight,0.000000,0.000,0.500000,0.350000,0.250000,0.000000,0.000000,0.000000,...,0,0,0,Orthodox,175.26,187.96,247.0,29.0,1.07,12.70


In [54]:
for col in b_fighter_avg2.columns:
    print(col)

B_fighter
weight_class
B_avg_KD
B_avg_opp_KD
B_avg_SIG_STR_pct
B_avg_opp_SIG_STR_pct
B_avg_TD_pct
B_avg_opp_TD_pct
B_avg_SUB_ATT
B_avg_opp_SUB_ATT
B_avg_REV
B_avg_opp_REV
B_avg_SIG_STR_att
B_avg_SIG_STR_landed
B_avg_opp_SIG_STR_att
B_avg_opp_SIG_STR_landed
B_avg_TOTAL_STR_att
B_avg_TOTAL_STR_landed
B_avg_opp_TOTAL_STR_att
B_avg_opp_TOTAL_STR_landed
B_avg_TD_att
B_avg_TD_landed
B_avg_opp_TD_att
B_avg_opp_TD_landed
B_avg_HEAD_att
B_avg_HEAD_landed
B_avg_opp_HEAD_att
B_avg_opp_HEAD_landed
B_avg_BODY_att
B_avg_BODY_landed
B_avg_opp_BODY_att
B_avg_opp_BODY_landed
B_avg_LEG_att
B_avg_LEG_landed
B_avg_opp_LEG_att
B_avg_opp_LEG_landed
B_avg_DISTANCE_att
B_avg_DISTANCE_landed
B_avg_opp_DISTANCE_att
B_avg_opp_DISTANCE_landed
B_avg_CLINCH_att
B_avg_CLINCH_landed
B_avg_opp_CLINCH_att
B_avg_opp_CLINCH_landed
B_avg_GROUND_att
B_avg_GROUND_landed
B_avg_opp_GROUND_att
B_avg_opp_GROUND_landed
B_avg_CTRL_time(seconds)
B_avg_opp_CTRL_time(seconds)
B_total_time_fought(seconds)
B_total_rounds_fought
B_tota

In [55]:
b_fighter_avg2[b_fighter_avg2['B_fighter'] == 'Grant Dawson']

,B_fighter,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,...,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,B_age,B_index_ape,B_index_ape2
5,Grant Dawson,Lightweight,0.0,0.0,0.515,0.47375,0.435,0.25,1.25,0.625,...,0,2,0,Switch,177.8,182.88,145.0,27.0,1.03,5.08
899,Grant Dawson,Featherweight,0.0,0.0,0.440,0.57000,0.540,0.00,1.00,1.000,...,0,0,0,Switch,177.8,182.88,145.0,25.0,1.03,5.08


In [56]:
x = b_fighter_avg2.groupby(['B_fighter', 'B_Stance', 'weight_class']).mean()
print(x)

                                              B_avg_KD  B_avg_opp_KD  \
B_fighter          B_Stance weight_class                               
Aaron Phillips     Southpaw Bantamweight      0.000000      0.000000   
Aaron Riley        Southpaw Lightweight       0.000000      0.213542   
                            Welterweight      0.000000      1.000000   
Aaron Rosa         Orthodox LightHeavyweight  0.000000      0.000000   
Aaron Simpson      Orthodox Middleweight      0.229167      0.000000   
...                                                ...           ...   
Zak Ottow          Orthodox Welterweight      0.145833      0.177083   
Zarrukh Adashev    Southpaw Flyweight         0.000000      1.000000   
Zelim Imadaev      Orthodox Welterweight      0.000000      0.500000   
Zhalgas Zhumagulov Switch   Flyweight         0.000000      0.000000   
Zubaira Tukhugov   Orthodox Featherweight     0.583333      0.000000   

                                              B_avg_SIG_STR_pct

In [57]:
y = b_fighter_avg2.groupby(['B_fighter', 'B_Stance', 'weight_class'])['weight_class'].count()
print(y)

B_fighter           B_Stance  weight_class    
Aaron Phillips      Southpaw  Bantamweight        1
Aaron Riley         Southpaw  Lightweight         3
                              Welterweight        1
Aaron Rosa          Orthodox  LightHeavyweight    1
Aaron Simpson       Orthodox  Middleweight        3
                                                 ..
Zak Ottow           Orthodox  Welterweight        6
Zarrukh Adashev     Southpaw  Flyweight           1
Zelim Imadaev       Orthodox  Welterweight        1
Zhalgas Zhumagulov  Switch    Flyweight           1
Zubaira Tukhugov    Orthodox  Featherweight       3
Name: weight_class, Length: 1326, dtype: int64


In [58]:
df_merged2 = x.merge(y, how = 'inner', left_index = True, right_index = True)

In [59]:
df_merged2 = df_merged2.rename(columns = {'weight_class' : 'count_weight_class'})

In [60]:
df_merged2 = df_merged2.reset_index()

In [61]:
b_df = df_merged2

In [62]:
b_df

,B_fighter,B_Stance,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,...,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_Height_cms,B_Reach_cms,B_Weight_lbs,B_age,B_index_ape2,count_weight_class
0,Aaron Phillips,Southpaw,Bantamweight,0.000000,0.000000,0.575000,0.585000,0.000000,0.640000,0.500000,...,0.000000,0.000000,0.0,0.0,175.26,180.34,135.0,30.000000,5.08,1
1,Aaron Riley,Southpaw,Lightweight,0.000000,0.213542,0.404010,0.379427,0.418438,0.220052,0.109375,...,2.000000,0.000000,0.0,0.0,172.72,175.26,155.0,28.666667,2.54,3
2,Aaron Riley,Southpaw,Welterweight,0.000000,1.000000,0.420000,0.410000,0.000000,0.750000,1.000000,...,0.000000,0.000000,0.0,0.0,172.72,175.26,155.0,25.000000,2.54,1
3,Aaron Rosa,Orthodox,LightHeavyweight,0.000000,0.000000,0.495000,0.470000,0.000000,0.175000,0.000000,...,0.000000,0.000000,0.0,0.0,193.04,195.58,205.0,28.000000,2.54,1
4,Aaron Simpson,Orthodox,Middleweight,0.229167,0.000000,0.489271,0.475625,0.505625,0.099167,0.125000,...,0.333333,2.000000,0.0,0.0,182.88,185.42,170.0,35.666667,2.54,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1321,Zak Ottow,Orthodox,Welterweight,0.145833,0.177083,0.378073,0.402943,0.123438,0.319219,0.367188,...,0.000000,0.333333,0.0,0.0,180.34,182.88,170.0,30.500000,2.54,6
1322,Zarrukh Adashev,Southpaw,Flyweight,0.000000,1.000000,0.330000,0.660000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,165.10,165.10,125.0,28.000000,0.00,1
1323,Zelim Imadaev,Orthodox,Welterweight,0.000000,0.500000,0.405000,0.555000,0.500000,0.330000,0.000000,...,0.000000,0.000000,0.0,0.0,182.88,193.04,170.0,25.000000,10.16,1
1324,Zhalgas Zhumagulov,Switch,Flyweight,0.000000,0.000000,0.550000,0.420000,0.180000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,162.56,167.64,125.0,32.000000,5.08,1


In [63]:
b_df.to_csv ('csv files/b_df.csv', index = False)

In [64]:
fighters = pd.read_csv('csv files/fighter_df.csv')

In [65]:
fighters

,FighterId,FirstName,LastName,Nickname,WeightClass,BirthDate,Height,Weight,Reach,Wins,...,CareerStats.FirstName,CareerStats.LastName,CareerStats.SigStrikesLandedPerMinute,CareerStats.SigStrikeAccuracy,CareerStats.TakedownAverage,CareerStats.SubmissionAverage,CareerStats.KnockoutPercentage,CareerStats.TechnicalKnockoutPercentage,CareerStats.DecisionPercentage,CareerStats
0,140000001,Chance,Rencountre,Black Eagle,Welterweight,1986-12-31T00:00:00,74.0,170.0,75.0,14.0,...,Chance,Rencountre,2.6,31.7,3.2,0.8,0.0,0.0,13.3,NaN
1,140000002,Kyle,Stewart,Gunz Up,Welterweight,1989-01-31T00:00:00,72.0,170.0,76.5,13.0,...,Kyle,Stewart,3.4,54.3,3.1,0.0,0.0,0.0,0.0,NaN
2,140000003,Geoff,Neal,Handz of Steel,Welterweight,1990-08-28T00:00:00,71.0,171.0,75.0,13.0,...,Geoff,Neal,10.5,80.4,1.4,0.5,36.3,36.3,14.0,NaN
3,140000004,Belal,Muhammad,Remember the Name,Welterweight,1988-07-09T00:00:00,71.0,170.0,72.0,19.0,...,Belal,Muhammad,11.0,77.4,4.3,0.4,10.4,10.4,53.7,NaN
4,140000005,Dennis,Bermudez,The Menace,Featherweight,1986-12-13T00:00:00,66.0,155.0,66.0,17.0,...,Dennis,Bermudez,10.6,70.8,9.1,2.4,8.8,8.8,40.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1158,140001170,Bruno,Korea,NaN,Flyweight,1991-06-28T00:00:00,66.0,125.0,65.5,12.0,...,Bruno,Korea,4.8,73.4,0.0,0.0,0.0,0.0,0.0,NaN
1159,140001171,C.J.,Vergara,C.J.,Bantamweight,1991-06-18T00:00:00,66.0,135.0,69.5,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1160,140001172,Javid,Basharat,The Snow Leopard,Bantamweight,NaN,67.0,135.0,0.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1161,140001173,Saimon,Oliveira,NaN,Bantamweight,1991-04-21T00:00:00,69.0,135.0,67.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
fighters.shape

(1163, 31)

In [67]:
for col in fighters.columns:
    print(col)

FighterId
FirstName
LastName
Nickname
WeightClass
BirthDate
Height
Weight
Reach
Wins
Losses
Draws
NoContests
TechnicalKnockouts
TechnicalKnockoutLosses
Submissions
SubmissionLosses
TitleWins
TitleLosses
TitleDraws
CareerStats.FighterId
CareerStats.FirstName
CareerStats.LastName
CareerStats.SigStrikesLandedPerMinute
CareerStats.SigStrikeAccuracy
CareerStats.TakedownAverage
CareerStats.SubmissionAverage
CareerStats.KnockoutPercentage
CareerStats.TechnicalKnockoutPercentage
CareerStats.DecisionPercentage
CareerStats


In [68]:
fighters.FighterId.value_counts()

140000001    1
140000802    1
140000782    1
140000781    1
140000780    1
            ..
140000388    1
140000387    1
140000386    1
140000385    1
140001174    1
Name: FighterId, Length: 1163, dtype: int64

In [69]:
fighter_weightclass = fighters[["WeightClass"]]

In [70]:
fighter_weightclass.value_counts()

WeightClass          
Lightweight              155
Welterweight             151
Featherweight            146
Bantamweight             134
Middleweight             111
Light Heavyweight         89
Heavyweight               87
Women's Strawweight       67
Women's Flyweight         57
Flyweight                 55
Women's Bantamweight      41
Women's Featherweight     10
Catch Weight               1
Catchweight                1
dtype: int64

In [71]:
indexname1 = fighters[fighters['WeightClass'] == "Women's Strawweight" ].index

In [72]:
fighters.drop(indexname1, inplace = True)

In [73]:
indexname2 = fighters[fighters['WeightClass'] == "Women's Flyweight"].index

In [74]:
fighters.drop(indexname2, inplace = True)

In [75]:
indexname3 = fighters[fighters['WeightClass'] == "Women's Bantamweight" ].index

In [76]:
fighters.drop(indexname3, inplace = True)

In [77]:
indexname4 = fighters[fighters['WeightClass'] == "Women's Featherweight" ].index

In [78]:
fighters.drop(indexname4, inplace = True)

In [79]:
fighters

,FighterId,FirstName,LastName,Nickname,WeightClass,BirthDate,Height,Weight,Reach,Wins,...,CareerStats.FirstName,CareerStats.LastName,CareerStats.SigStrikesLandedPerMinute,CareerStats.SigStrikeAccuracy,CareerStats.TakedownAverage,CareerStats.SubmissionAverage,CareerStats.KnockoutPercentage,CareerStats.TechnicalKnockoutPercentage,CareerStats.DecisionPercentage,CareerStats
0,140000001,Chance,Rencountre,Black Eagle,Welterweight,1986-12-31T00:00:00,74.0,170.0,75.0,14.0,...,Chance,Rencountre,2.6,31.7,3.2,0.8,0.0,0.0,13.3,NaN
1,140000002,Kyle,Stewart,Gunz Up,Welterweight,1989-01-31T00:00:00,72.0,170.0,76.5,13.0,...,Kyle,Stewart,3.4,54.3,3.1,0.0,0.0,0.0,0.0,NaN
2,140000003,Geoff,Neal,Handz of Steel,Welterweight,1990-08-28T00:00:00,71.0,171.0,75.0,13.0,...,Geoff,Neal,10.5,80.4,1.4,0.5,36.3,36.3,14.0,NaN
3,140000004,Belal,Muhammad,Remember the Name,Welterweight,1988-07-09T00:00:00,71.0,170.0,72.0,19.0,...,Belal,Muhammad,11.0,77.4,4.3,0.4,10.4,10.4,53.7,NaN
4,140000005,Dennis,Bermudez,The Menace,Featherweight,1986-12-13T00:00:00,66.0,155.0,66.0,17.0,...,Dennis,Bermudez,10.6,70.8,9.1,2.4,8.8,8.8,40.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1158,140001170,Bruno,Korea,NaN,Flyweight,1991-06-28T00:00:00,66.0,125.0,65.5,12.0,...,Bruno,Korea,4.8,73.4,0.0,0.0,0.0,0.0,0.0,NaN
1159,140001171,C.J.,Vergara,C.J.,Bantamweight,1991-06-18T00:00:00,66.0,135.0,69.5,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1160,140001172,Javid,Basharat,The Snow Leopard,Bantamweight,NaN,67.0,135.0,0.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1161,140001173,Saimon,Oliveira,NaN,Bantamweight,1991-04-21T00:00:00,69.0,135.0,67.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
fighters2 = fighters.drop(columns = ['CareerStats'])

In [81]:
fighters2 = fighters2.dropna()

In [82]:
fighters2

,FighterId,FirstName,LastName,Nickname,WeightClass,BirthDate,Height,Weight,Reach,Wins,...,CareerStats.FighterId,CareerStats.FirstName,CareerStats.LastName,CareerStats.SigStrikesLandedPerMinute,CareerStats.SigStrikeAccuracy,CareerStats.TakedownAverage,CareerStats.SubmissionAverage,CareerStats.KnockoutPercentage,CareerStats.TechnicalKnockoutPercentage,CareerStats.DecisionPercentage
0,140000001,Chance,Rencountre,Black Eagle,Welterweight,1986-12-31T00:00:00,74.0,170.0,75.0,14.0,...,140000001.0,Chance,Rencountre,2.6,31.7,3.2,0.8,0.0,0.0,13.3
1,140000002,Kyle,Stewart,Gunz Up,Welterweight,1989-01-31T00:00:00,72.0,170.0,76.5,13.0,...,140000002.0,Kyle,Stewart,3.4,54.3,3.1,0.0,0.0,0.0,0.0
2,140000003,Geoff,Neal,Handz of Steel,Welterweight,1990-08-28T00:00:00,71.0,171.0,75.0,13.0,...,140000003.0,Geoff,Neal,10.5,80.4,1.4,0.5,36.3,36.3,14.0
3,140000004,Belal,Muhammad,Remember the Name,Welterweight,1988-07-09T00:00:00,71.0,170.0,72.0,19.0,...,140000004.0,Belal,Muhammad,11.0,77.4,4.3,0.4,10.4,10.4,53.7
4,140000005,Dennis,Bermudez,The Menace,Featherweight,1986-12-13T00:00:00,66.0,155.0,66.0,17.0,...,140000005.0,Dennis,Bermudez,10.6,70.8,9.1,2.4,8.8,8.8,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,140001122,Shayilan,Nuerdanbieke,Wolverine,Featherweight,1994-05-04T00:00:00,68.0,146.0,69.0,19.0,...,140001122.0,Shayilan,Nuerdanbieke,2.7,50.0,4.8,0.0,0.0,0.0,0.0
1112,140001124,Chris,Barnett,Beastboy,Heavyweight,1986-06-14T00:00:00,69.0,263.0,75.0,21.0,...,140001124.0,Chris,Barnett,11.1,61.0,0.0,0.0,0.0,0.0,0.0
1113,140001125,Juancamilo,Ronderos,100,Flyweight,1995-02-17T00:00:00,63.0,128.0,64.0,4.0,...,140001125.0,Juancamilo,Ronderos,11.4,56.1,0.0,0.0,0.0,0.0,0.0
1120,140001132,Preston,Parsons,Pressure,Welterweight,1995-07-15T00:00:00,71.0,169.0,71.0,9.0,...,140001132.0,Preston,Parsons,14.5,84.2,0.0,0.0,0.0,0.0,0.0


In [83]:
fighters2.to_csv ('csv files/fighters2.csv', index = False)

In [84]:
fighter_weightclass = fighters[["WeightClass"]]

In [85]:
fighter_weightclass.value_counts()

WeightClass      
Lightweight          155
Welterweight         151
Featherweight        146
Bantamweight         134
Middleweight         111
Light Heavyweight     89
Heavyweight           87
Flyweight             55
Catch Weight           1
Catchweight            1
dtype: int64

In [86]:
fighter_weightclass.to_csv ('csv files/fighter_weightclass.csv', index = False)